# Exercise 1

Take the titanic dataset and using all attributes to predict the class `Survived` (convert age and fare into classes ; exclude names from the attribute list)

In [1]:
% load_ext autoreload
% autoreload 2
% matplotlib inline

UsageError: Line magic function `%` not found.


In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
titanic = pd.read_csv('../Data/titanic.csv.zst', index_col='Name')

titanic.describe(include='all')

First we create classes and label them.

In [ ]:
from sklearn import preprocessing

titanic['Age Group'] = pd.qcut(x=titanic['Age'], q=4)
titanic['Fare Group'] = pd.qcut(x=titanic['Fare'], q=4)

for col in ['Sex', 'Age Group', 'Fare Group']:
    le = preprocessing.LabelEncoder()
    titanic[col] = le.fit_transform(titanic[col])

titanic.head(5)

Some preliminary definitions to use later.

In [ ]:
all_features = ['Pclass', 'Sex', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Age Group', 'Fare Group']
max_depth = len(all_features) + 1

In [ ]:
## (a) Choose Three classifiers and evaluate their performance using all attributes

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

models = {
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(metric='canberra'),
    # Note: We use the `canberra` metric, as it has proven to be the best one in the past exercises.
    'Naïve Bayes': GaussianNB(),
}

In [ ]:
from util import *

for name, model in models.items():
    print(f'{model}:')
    a, p, r, f = kfold_eval(model, titanic[all_features], titanic['Survived'], 5)
    data = {
        'Fold': range(1, 6),
        'Accuracy': a,
        'Precision': p,
        'Recall': r,
        'F1-Score': f,
    }

    scores = pd.DataFrame(data).set_index('Fold')
    display(scores)

## (b) Define a feature selection method and use it on all the classifiers;
We will be using PCA for this task.

In [ ]:
from pca_eval import *

for name, model in models.items():
    a, p, r, f1 = pca_eval(model, titanic[all_features], titanic['Survived'], k=5, c=4)

    print(f'{name}:')
    data = {
        'Fold': range(1, 6),
        'Accuracy': a,
        'Precision': p,
        'Recall': r,
        'F1-Score': f,
    }

    scores = pd.DataFrame(data).set_index('Fold')
    display(scores)

## (c) Compare the classifiers and explain the differences observed;

PCA is a data reduction technique compressing all features into just a few principal components.

The performance seems to be consistently lower though.
Further checking/changing code did not shine light on as to why.